In [77]:
import sqlite3 as sql
import pandas as pd
from IPython.display import Image

In [2]:
df = pd.read_csv('./database.sqlite')

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- CASEID - case number
- DOF1 - direction of force (highest)
- GAD1 - deformation location (highest)
- PSU - primary sampling unit number
- YEAR - calender year of crash
- STRATIF - case stratum
- BODYTYPE - vehicle body type
- DVEST - estimated highest delta V
- DVBASIS - basis for total deltav V (highest)
- DVTOTAL - total delta V
- MAKE - vehicle make
- MODELYR - vehicle model year
- AGE - age of occupant
- BAGAVAIL - air bag system availability
- BAGDEPLY - air bag system deployed
- BAGFAIL - air bag system failure
- CASENO - case sequence number
- CHTYPE - type of child safety seat
- DEATH - time of death
- HEIGHT - height of occupent
- HOPSTAY - hospital stay
- INJSEV - injury severity (police rating)
- MANAVAIL -
- MANFAIL - 
- MANUSE - manual belt system use 
- OCCNO - occupant number
- ROLE - occupant's role
- SEATPOS - occupant's seat position
- SEX - occupant's sex
- TREATMENT - treatment - mortality
- WEIGHT - occupant's weight

In [19]:
fatal = df.copy()

In [20]:
fatal.head()

,Unnamed: 0,CASEID,DOF1,GAD1,PSU,YEAR,STRATIF,BODYTYPE,DVEST,DVBASIS,...,INJSEV,MANAVAIL,MANFAIL,MANUSE,OCCNO,ROLE,SEATPOS,SEX,TREATMNT,WEIGHT
0,1,1H,NaN,F,2,2004,H,4.0,6.0,11.0,...,0.0,4.0,1.0,4.0,1.0,1.0,11.0,1.0,0.0,73.0
1,2,1H,NaN,F,2,2004,H,4.0,6.0,11.0,...,0.0,4.0,1.0,4.0,1.0,1.0,11.0,1.0,0.0,73.0
2,3,1H,NaN,F,2,2004,H,4.0,6.0,11.0,...,0.0,4.0,1.0,4.0,1.0,1.0,11.0,1.0,0.0,73.0
3,4,1H,NaN,F,2,2004,H,4.0,6.0,11.0,...,0.0,4.0,1.0,4.0,1.0,1.0,11.0,1.0,0.0,73.0
4,5,1H,NaN,F,2,2004,H,4.0,6.0,11.0,...,0.0,4.0,1.0,4.0,1.0,1.0,11.0,1.0,0.0,73.0


In [21]:
# changing all columns to lower case
columns = fatal.columns
columns_lower = []
for col in columns:
    columns_lower.append(col.lower())
fatal.columns = columns_lower

In [22]:
## removed unnamed: 0 column since it matched the index
fatal.drop(labels='unnamed: 0',axis=1,inplace=True)

In [62]:
# dictionay of categorie ID's (named how they will appear when get dummies is applied)
dvest = {'dvest_0':'reconstructed_dv','dvest_1':'dv<10kmph','dvest_2':'10 >= dv < 25kmph','dvest_3':'25 >= dv < 40kmph',
         'divest_4':'40 >= dv <55kmph','divest_5':'dv >= 55kmph','divest_6':'minor damage','divest_7':'moderat damage',
         'divest_8':'severe damage'}

dvbasis = {'dvbasis_0':'not_ispctd','dvbasis_1':'S_dmg_only','dvbasis_2':'S_dmg_&_trjctry','dvbasis_3':'S_missing_vhcl', 
           'dvbasis_4':'S_dmg_cdc_only', 'dvbasis_5':'one_vhcle_byond_S_scope','dvbasis_6':'rollover',
           'dvbasis_7':'non_hrzntl_forces','dvbasis_8':'sideswipe_dmg','dvbasis_9':'severe_override',
          'dvbasis_10':'ylding_objct','dvbasis_11':'overlapping_dmg','dvbasis_12':'insuf_data',
           'dvbasis_98':'other','dvbasis_99':'unknown'}

gad1 = {'gad1_F':'front','gad1_L':'left','gad_R':'right','gad1_B':'back','gad1_uk':'unkown','gad1_T':'top',
        'gad1_U':'undercarriage'}


bagavail = {'bagavail_1':'no','bagavail_2':'yes','bagavail_3':'unknown'}
# stratif = {'stratif_k':'fatality','stratif_F':'injry_no_tow_hsptl',
#           'stratif_E':'injry_tow','stratif_C':'two_tow_no_hsptl','stratif_D':}
# air bad deployed 
bagdeply = {'bagdeply_0':'no_bag', 'bagdeply_1':'impct_deply','bagdeply_3':'deply_ukdetls',
            'bagdeply_5':'uk_deply','bagdeply_7':'no_deply'}

# air bag failure
bagfail = {'bagfail_0':'no_bag','bagfail_1':'no','bagfail_2':'yes'}

# child seat type
chtype = {'chtype chtype _0':'none','chtype_1':'conv_seat','chtype_2':'frwd_face','chtype_3':'infnt_seat',
          'chtype_4':'boost_seat','chtype_7':'vest','chtype_8':'harness'}

# Injury severity
injsev = {'injsev_0':'no_inj','injsev_1':'pssbl_inj','injsev_2':'n_incptng_inj','injsev_3':'incptng_inj',
          'injsev_4':'killed','injsev_5':'inj_sev_uk','injsev_6':'die_b4_crsh'}

# Drop mannavail since it's same data as manuse
# Manual seatbelt used
manuse = {'manuse_0':'none','manuse_1':'inoperative','manuse_2':'shldr_b','manuse_3':'lap_b',
          'manuse_4':'laphshldr_b','manuse_5':'b_avail_uk','manuse_8':'othr_b_use',
          'manuse_12':'shldr_chld_seat','manuse_13':'lap_chld_seat','manuse_14':'lapshldr_chld_seat',
          'manuse_15':'b_chld_seat_uk','manuse_18':'othrb_chld_seat'}

# Manual seatbelt fail
manfail = {'manfail_0':'nouse','manfail_1':'no_mlfctn',
           'manfail_2':'torn_wbbng','manfail_3':'bckl',
           'manfail_4':'uppr_anchr','manfail_5':'othr_anchr',
           'manavail_6':'retrctr','manavail_7':'combo',
           'manfail_8':'other'}

role = {'role_1':'driver','role_2':'passenger'}

# occupant number mix of occno(11- 19) & posture (1-10)
occno = {'occno_0':'nrml_pstr','occno_1':'kneelng_stndng','occno_2':'lyng_accss_seat',
        'occno_3':'kneelng_stndng_frnt_seat','occno_4'}



In [24]:
# replace 9 with uk for (unknown per the NASS documents)
fatal['gad1'] = fatal['gad1'].copy().replace('9','uk')

In [25]:
# replace 0 with 1 since they are the same thing according to the NASS documents
fatal['bagavail'] = fatal['bagavail'].copy().replace(0,1,axis=0)

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/pandas/core/generic.py:4480: UserWarning: the "axis" argument is deprecated and will be removed inv0.13; this argument has no effect
  warnings.warn('the "axis" argument is deprecated '


In [26]:
# removing letter at the end and going to drop caseno and PSU since 
# they "are identical variables across all NASS records"
fatal['caseid'] = fatal['caseid'].apply(lambda x: x[:-1].strip())

In [27]:
# according NASS these columns are same info as caseid
fatal.drop(['psu','caseno'],axis=1,inplace=True)

In [65]:
# remove manavail since manuse gives the same information
fatal.drop('manavail', axis=1, inplace=True)

In [68]:
# remove bagavail since bagdeply has the same information
fatal.drop('bagavail', axis=1, inplace=True)

In [69]:
# Dictionary of each column's unique values key = counts, value = feature name
unique_dict = {}
for col in fatal.columns:
    unique_dict[col] =  fatal[col].nunique()

# Features <30 couts
categoricals = []
for key in unique_dict.keys():
    if unique_dict[key] < 30:
        categoricals.append(key)

In [70]:
for col in categoricals:
    print(col)
    print(fatal[col].value_counts())

gad1
F     1372618
L      411350
B      330703
R      319169
uk     100183
T       65536
U        3328
Name: gad1, dtype: int64
year
2004    544672
2006    500428
2005    494713
2007    243975
2008    195223
2009    181650
2010    157642
2011    150124
2012    104230
2014    103450
2013     22257
2015     18063
Name: year, dtype: int64
stratif
K    529817
F    442408
E    385249
C    236676
D    235326
G    230456
B    226083
H    214204
J    156951
A     59252
Name: stratif, dtype: int64
dvest
0.0    1674684
7.0     306520
6.0     131007
8.0      84874
2.0      12325
3.0       5534
1.0       3007
5.0       2872
4.0       1199
Name: dvest, dtype: int64
dvbasis
1.0     1035657
3.0      587972
11.0     301492
0.0      295200
4.0      109304
7.0       66250
5.0       53107
10.0      49190
98.0      40293
6.0       37035
9.0       32346
8.0       17075
2.0        1522
Name: dvbasis, dtype: int64
bagdeply
1.0    801298
7.0    779497
0.0    673300
5.0     32038
3.0     26686
Name: bagdeply, 

In [64]:
fatal.dtypes

caseid       object
dof1        float64
gad1         object
year          int64
stratif      object
bodytype    float64
dvest       float64
dvbasis     float64
dvtotal     float64
make        float64
modelyr     float64
age         float64
bagavail    float64
bagdeply    float64
bagfail     float64
chtype      float64
death       float64
height      float64
hospstay    float64
injsev      float64
manavail    float64
manfail     float64
manuse      float64
occno       float64
role        float64
seatpos     float64
sex         float64
treatmnt    float64
weight      float64
dtype: object

In [30]:
fatal.isnull().sum()

caseid            0
dof1         299352
gad1         113540
year              0
stratif           5
bodytype       2519
dvest        494405
dvbasis       89984
dvtotal     1091276
make          10659
modelyr       13077
age           56349
bagavail     403608
bagdeply     403608
bagfail      576251
chtype       284986
death        327097
height       689620
hospstay     394158
injsev        49824
manavail     411021
manfail      680669
manuse       495745
occno           165
role            200
seatpos      190062
sex           32035
treatmnt     331950
weight       654910
dtype: int64

In [32]:
# Imputer

In [33]:
fatal.isnull().sum()

caseid            0
dof1         299352
gad1         113540
year              0
stratif           5
bodytype       2519
dvest        494405
dvbasis       89984
dvtotal     1091276
make          10659
modelyr       13077
age           56349
bagavail     403608
bagdeply     403608
bagfail      576251
chtype       284986
death        327097
height       689620
hospstay     394158
injsev        49824
manavail     411021
manfail      680669
manuse       495745
occno           165
role            200
seatpos      190062
sex           32035
treatmnt     331950
weight       654910
dtype: int64

In [76]:
fatal[ fatal['occno'] == 12 ]['year'].value_counts()

2009    256
2006    156
2004    136
2005     34
2011     30
2010      5
2013      4
Name: year, dtype: int64

In [43]:
dvtotal_na = fatal[ fatal['dvtotal'].isna()]['year'].value_counts()